# CMIP6 Exogenous Variables for GRACE 5y forecast (Climate Models MPI)

This script processes local NetCDF files containing climate data by clipping them to basin geometries, calculating the average value within each basin, and, if necessary, finding the nearest grid point. The resulting time series for each basin is saved as a CSV file.

#### Loading necessary libraries

In [67]:
import pandas as pd
import geopandas as gpd

# Libraries for working with multidimensional arrays
import numpy as np
import xarray as xr

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# More libraries
import xarray as xr
import dask
import gc
import os
import rioxarray
from shapely import wkt
from scipy.spatial import cKDTree 

In [68]:
# Instruction to show all rows and columns when printing dataframes 
pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

In [ ]:
# Initialize the directory containing local NetCDF files
data_directory = 'CMIP6_downloads'

# Function to find the nearest point on the lat/lon grid to a given basin geometry
def find_nearest(lat, lon, basin_geom):
    """
    Find the nearest point on the lat/lon grid to the centroid of a basin geometry.
    """
    latlon_points = [(lt, ln) for lt in lat for ln in lon]  # Create a list of lat/lon point pairs
    tree = cKDTree(latlon_points)  # Use a KDTree for efficient nearest-neighbor search
    basin_center = basin_geom.centroid  # Get the centroid of the basin
    dist, idx = tree.query([basin_center.y, basin_center.x])  # Find the nearest point in the grid
    nearest_point = latlon_points[idx]
    return nearest_point

# Function to process a local NetCDF file for a specific climate variable
def process_local_nc_file(filepath, gdf):
    """
    Processes a local NetCDF file, extracts climate data, and clips it to basin geometries.
    
    Parameters:
    - filepath (str): Path to the NetCDF file.
    - gdf (GeoDataFrame): GeoDataFrame with basin geometries.
    
    Returns:
    - DataFrame: Basin time series data with corresponding year and month.
    """
    
    # 1. Load the dataset from the NetCDF file
    exog_var = xr.open_dataset(filepath)
    
    # 2. Select the last variable available in the dataset
    last_variable = list(exog_var.data_vars.keys())[-1]
    exog_var = exog_var[[last_variable]]
    print(f"Processing variable '{last_variable}' from {filepath}")

    # 3. Filter the dataset for dates after January 2015
    exog_var = exog_var.sel(time=slice("2015-01-01", None))
    
    # 4. Ensure the dataset has the correct CRS and apply chunking for latitude, longitude, and time
    exog_var = exog_var.rio.write_crs("EPSG:4326")
    exog_var = exog_var.chunk({'lat': 50, 'lon': 50, 'time': 50})

    # 5. Initialize a list to store Dask tasks for parallel processing
    tasks = []
    
    # 6. Function to process each basin and clip the exog_var dataset
    @dask.delayed
    def process_basin(geom, basin_id):
        # Check if the grid points fall within the basin
        mask = exog_var.rio.clip([geom], exog_var.rio.crs, drop=False)
        
        if mask[last_variable].notnull().any():
            # If grid points exist within the basin, average the values
            basin_series = mask[last_variable].mean(dim=['lat', 'lon']).compute()
        else:
            # If no points exist, find the nearest grid point to the basin
            nearest_lat, nearest_lon = find_nearest(exog_var['lat'].values, exog_var['lon'].values, geom)
            nearest_point = exog_var.sel(lat=nearest_lat, lon=nearest_lon)
            basin_series = nearest_point[last_variable].compute()
        
        return pd.Series(basin_series.values, name=basin_id)
    
    # 7. Loop through the basin geometries and apply the processing function
    for _, row in gdf.iterrows():
        task = process_basin(row['geometry'], row['basin_id'])  # Create the delayed task
        tasks.append(task)  # Add the task to the list
    
    # 8. Compute all tasks in parallel using Dask
    basin_time_series = dask.compute(*tasks)
    
    # 9. Create a DataFrame from the basin time series
    basin_time_series_df = pd.concat(basin_time_series, axis=1)
    basin_time_series_df['datetime'] = pd.to_datetime(exog_var['time'].values)  # Convert time to datetime
    
    # Extract year and month from the datetime column
    basin_time_series_df['Year'] = basin_time_series_df['datetime'].dt.year
    basin_time_series_df['Month'] = basin_time_series_df['datetime'].dt.month

    return basin_time_series_df

# Main function to process all NetCDF files in the directory
def process_all_files(gdf):
    """
    Processes all NetCDF files in the specified directory, extracting and aggregating 
    time series data for each basin.
    
    Parameters:
    - gdf (GeoDataFrame): GeoDataFrame with basin geometries.
    
    Returns:
    - DataFrame: Combined time series data for all basins.
    """
    all_basin_series = pd.DataFrame()

    for filename in os.listdir(data_directory):
        if filename.endswith(".nc"):
            filepath = os.path.join(data_directory, filename)
            basin_series_df = process_local_nc_file(filepath, gdf)
            all_basin_series = pd.concat([all_basin_series, basin_series_df], axis=1)
    
    # Clean up memory
    gc.collect()
    
    return all_basin_series

# Load basin information from a CSV file and convert it to a GeoDataFrame
gdf = gpd.read_file('basin_info_df.csv')
gdf['geometry'] = gdf['geometry'].apply(wkt.loads)  # Load WKT geometries
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
gdf.set_crs("EPSG:4326", inplace=True)  # Set CRS if not defined

# Process all NetCDF files in the local directory
basin_time_series = process_all_files(gdf)

# Save the processed time series data to a CSV file
basin_time_series.to_csv('processed_basin_time_series.csv', index=False)

print("Process completed successfully.")

In [1]:
### Dataframe is already saved